In [ ]:
!pip install -q langchain-text-splitters==0.3.5
!pip install -qU langchain-openai
!pip install -qU "langchain-chroma>=0.1.2"

## 1.Get Channel Subtitle

for the demo we are using popular 10 vidoes from this channel
`https://www.youtube.com/@TEDx`

In [1]:
directory = "channel_data"

In [3]:
from ytkit import GetChannelSubtitle

get_channel_subtlte = GetChannelSubtitle()
get_channel_subtlte.get(channel_username="TEDx",
                        sort_by="popular",
                        limit=10,
                        language="en",
                        save_to_disk=directory,
                        )

100%|██████████| 10/10 [01:05<00:00,  6.55s/it]


## 2. Splitting-Text

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os



# Get all files from the directory
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
texts=[]
for file in files:
    # Load example document
    with open(f"{directory}/{file}") as f:
        text = f.read()
        texts.append(text)

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=400,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents(texts)

In [5]:
texts[:3]

[Document(metadata={}, page_content="Translator: Vicky Lara Labaila\nReviewer: Denise RQ It's been said if you never want\nto have sex again, get married. (Laughter) Sex is one of the most contentious\nissues in marriage today, second only to finances. There are many married couples that have not had sex\nfor months, even years, and that's OK as long as they are OK\nwith it and happily married. The problem arises when one person"),
 Document(metadata={}, page_content="in the relationship wants sex, and the other doesn't. According to a national newspaper survey\nof approximately 10,000 respondents - mostly married men - 75% were satisfied in their relationship, but more than 50%\nwere dissatisfied with their sex lives. We are having sex\n- don't get me wrong - we are just having sex at the wrong time,"),
 Document(metadata={}, page_content="and with the wrong people; I'll get to that later. (Laughter) We are having sex before we get married,\nten years on average, so we are effectively

## 3. Ingesting in vectorDB

In [6]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [7]:
from langchain_chroma import Chroma

vector_store = Chroma(
    embedding_function=embeddings,
)
vector_store.add_documents(documents=texts)

['6115f44b-ffdf-4e53-8b1a-aa970bb8ca81',
 '7b299136-307b-4ad7-ad50-ccdd326a629b',
 'd30bc203-84a3-45d6-ae16-73e8de8f13bc',
 'faa3054f-a0aa-4fe2-8648-77b89066465e',
 'caaed9ec-2996-41f1-a3a2-629aaee57969',
 '48b4decf-7989-4441-9150-36842026c826',
 '1b212de7-35ba-42ec-876d-96f49ee2d89d',
 'c598be12-252a-42dc-a259-e7fc2988595d',
 '79e42deb-96ce-424f-8794-2c599ca363e5',
 'f8c06bd2-17da-48ef-8d6e-e627adec91f6',
 '01ee0467-5d64-4448-a52b-d834849fbe2c',
 '65c67917-1dc9-440c-875b-c0a5a9e05692',
 '6f385fb0-6aa3-4a3f-a16c-5484140e5028',
 'a1cc1796-fdc9-4f78-bad9-52643e4f2ae4',
 '921d0b7c-ee77-4050-9d1d-18641b235db2',
 '6dd27299-2c19-4143-bfd0-3a0b0cbdf14e',
 'fd879b82-16fd-4e81-9b10-97f7d24f59c3',
 '556c8046-a8e7-4960-874e-987a8f81e27c',
 '29873f34-1308-4f70-b0d5-9a5bb47fb417',
 '5980c198-72d4-44c5-9308-aa719d8fb0c0',
 'a57c91d6-0b01-4bea-a2b9-6ea880401748',
 '34af1f5c-f53a-486e-829d-c802133c4c4b',
 'aec04566-68d8-43bf-8a43-54fb4ce8e5e2',
 '92f3d9fa-e5b9-4550-948f-f0badd066812',
 '6feb7f70-c99a-

In [8]:
query = " philosophy for a happy life"
retriever = vector_store.as_retriever(
            search_type="similarity", search_kwargs={"k": 2}
        )
relevant_docs = retriever.invoke(query)
relevant_docs

[Document(id='95a54a6b-b317-40ed-b0ed-3d44714e1e77', metadata={}, page_content='most of them anyway. So I’m here today, to share with you\nmy philosophy for a happy life. So, for me,\nthere are 3 aspects to this philosophy. So this is a quote\nfrom the famous Ferris Bueller. The first aspect to my philosophy is that I’m okay with what I ultimately can’t do because there is so much I can do. Now people sometimes'),
 Document(id='0319514f-1ef7-42d6-bc5c-2bdf4934a8b1', metadata={}, page_content='that I want to be with, and I keep moving forward. So with this philosophy,\nI hope that all of you, regardless of your obstacles, can have a very happy life as well. Oh, wait, hang on a second, one more piece of advice –- (Laughter) Never miss a party if you can help it. My school’s homecoming dance\nis tomorrow night, and I will be there. Thank you very much. (Applause)')]

## 4. Inference

In [9]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
model = ChatOpenAI(model="gpt-4o-mini")

/tmp/ipykernel_15731/3798706509.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model="gpt-4o-mini")


In [10]:
from langchain import hub
prompt = hub.pull("empler-ai/rag-prompt")

/home/rahul/anaconda3/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
writer = prompt | model | StrOutputParser()

In [12]:
content=[doc.page_content for doc in relevant_docs]
context = "\n\n".join(content)

In [13]:
query="philosophy for a happy life"

In [14]:
answer=writer.invoke({"question":query,"context":context})

In [15]:
answer

"The philosophy for a happy life shared in the context emphasizes three aspects, with the first being acceptance of one's limitations while appreciating the multitude of possibilities available. The speaker conveys that it's important to focus on what can be done rather than what cannot, and encourages others to move forward despite obstacles. Additionally, there's a lighthearted reminder to enjoy life and not miss out on social gatherings, suggesting that connection and celebration are key components of happiness."